In [77]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [78]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

# Titanic Survivor Prediction with Machine Learning

## Objective:

In this notebook my objective is to find the best performing  machine learning model for predicting the survial of passengers based on the information given in the dataset.

## Data Cleaning:

**Have a look at the train dataset**

In [79]:
train_data=pd.read_csv('/kaggle/input/titanic/train.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


**Look at the information of the dataframe**

In [80]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


We can see that there are many data points missing from the features 'Age' and 'Cabin'.

In [81]:
test_data=pd.read_csv('/kaggle/input/titanic/test.csv')
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [82]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


Here also we can observe the same scenario.

So, we have to replace the missing values either with mean or frequency. Let's describe the data set find the mean and frequency of each feature

In [83]:
train_data.describe(include = 'all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",male,NaN,NaN,NaN,347082,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [84]:
train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)
test_data['Age'].fillna(test_data['Age'].mean(), inplace=True)

test_data["Fare"].fillna(test_data['Fare'].mean(), inplace=True)

train_data["Embarked"].fillna("S", inplace = True) 

In [85]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [86]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [87]:
train_data['Age'].describe()

count    891.000000
mean      29.699118
std       13.002015
min        0.420000
25%       22.000000
50%       29.699118
75%       35.000000
max       80.000000
Name: Age, dtype: float64

Lets divide the Age into 10 age groups

In [88]:
bins = np.linspace(min(train_data["Age"]), max(train_data["Age"]), 11)
bins
group_names = ['8', '16', '24', '32','40','48','56','64', '72','80']

In [89]:
train_data['Age-binned'] = pd.cut(train_data['Age'], bins, labels=group_names, include_lowest=True)
train_data[['Age','Age-binned']].head(20)

,Age,Age-binned
0,22.000000,24
1,38.000000,40
2,26.000000,32
3,35.000000,40
4,35.000000,40
5,29.699118,32
6,54.000000,56
7,2.000000,8
8,27.000000,32
9,14.000000,16


In [90]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    int64   
 2   Pclass       891 non-null    int64   
 3   Name         891 non-null    object  
 4   Sex          891 non-null    object  
 5   Age          891 non-null    float64 
 6   SibSp        891 non-null    int64   
 7   Parch        891 non-null    int64   
 8   Ticket       891 non-null    object  
 9   Fare         891 non-null    float64 
 10  Cabin        204 non-null    object  
 11  Embarked     891 non-null    object  
 12  Age-binned   891 non-null    category
dtypes: category(1), float64(2), int64(5), object(5)
memory usage: 84.9+ KB


In [91]:
test_data['Age-binned'] = pd.cut(test_data['Age'], bins, labels=group_names, include_lowest=True )
test_data[['Age','Age-binned']].head(20)

,Age,Age-binned
0,34.50000,40
1,47.00000,48
2,62.00000,64
3,27.00000,32
4,22.00000,24
5,14.00000,16
6,30.00000,32
7,26.00000,32
8,18.00000,24
9,21.00000,24


similarly divide the Fare into 6 bins.

In [92]:
bins1 = np.linspace(min(train_data["Fare"]), max(train_data["Fare"]), 7)
group_names1 = ['1', '2', '3', '4','5','6']

In [93]:
train_data['Fare-binned'] = pd.cut(train_data['Fare'], bins1, labels=group_names1, include_lowest=True )
train_data[['Fare-binned']].value_counts()

Fare-binned
1              825
2               46
3               11
4                6
6                3
5                0
dtype: int64

In [94]:
test_data['Fare-binned'] = pd.cut(test_data['Fare'], bins1, labels=group_names1, include_lowest=True )
test_data[['Fare-binned']].value_counts()

Fare-binned
1              384
2               16
3               10
4                7
6                1
5                0
dtype: int64

So far I have cleaned both the datasets given for the competetion. Now I want to use **train_data** for training and validation of my models and then I will use the best performing model on the **test_data**

In [95]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    int64   
 2   Pclass       891 non-null    int64   
 3   Name         891 non-null    object  
 4   Sex          891 non-null    object  
 5   Age          891 non-null    float64 
 6   SibSp        891 non-null    int64   
 7   Parch        891 non-null    int64   
 8   Ticket       891 non-null    object  
 9   Fare         891 non-null    float64 
 10  Cabin        204 non-null    object  
 11  Embarked     891 non-null    object  
 12  Age-binned   891 non-null    category
 13  Fare-binned  891 non-null    category
dtypes: category(2), float64(2), int64(5), object(5)
memory usage: 86.0+ KB


I'm ignoring the *PassengerID, Name* as they are unique values, hence they will not be useful to build a machine learning model. I'm also ignoring *Cabin* as majority of the entries are missing for this feature.

In [96]:
features = ["Pclass", "Sex", "SibSp", "Parch","Age-binned","Fare-binned", "Embarked", "Survived"]

In [97]:
df = train_data[features]
df.head()

,Pclass,Sex,SibSp,Parch,Age-binned,Fare-binned,Embarked,Survived
0,3,male,1,0,24,1,S,0
1,1,female,1,0,40,1,C,1
2,3,female,0,0,32,1,S,1
3,1,female,1,0,40,1,S,1
4,3,male,0,0,40,1,S,0


In [98]:
df = pd.get_dummies(df)
df.head()

,Pclass,SibSp,Parch,Survived,Sex_female,Sex_male,Age-binned_8,Age-binned_16,Age-binned_24,Age-binned_32,...,Age-binned_80,Fare-binned_1,Fare-binned_2,Fare-binned_3,Fare-binned_4,Fare-binned_5,Fare-binned_6,Embarked_C,Embarked_Q,Embarked_S
0,3,1,0,0,0,1,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
1,1,1,0,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,3,0,0,1,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
3,1,1,0,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,3,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [99]:
feature_cols = [x for x in df.columns if x != 'Survived']
feature_cols


['Pclass',
 'SibSp',
 'Parch',
 'Sex_female',
 'Sex_male',
 'Age-binned_8',
 'Age-binned_16',
 'Age-binned_24',
 'Age-binned_32',
 'Age-binned_40',
 'Age-binned_48',
 'Age-binned_56',
 'Age-binned_64',
 'Age-binned_72',
 'Age-binned_80',
 'Fare-binned_1',
 'Fare-binned_2',
 'Fare-binned_3',
 'Fare-binned_4',
 'Fare-binned_5',
 'Fare-binned_6',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S']

## Split the data:

I'm the splitting the train_data into two sets as I want to use one test for training and the the other for the validation of different models.

In [100]:
df.Survived.value_counts(normalize = True)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

As the target is not balanced, it is better to use StratifiedShuffleSplit to split the data

In [101]:
from sklearn.model_selection import StratifiedShuffleSplit

# Get the split indexes
strat_shuf_split = StratifiedShuffleSplit(n_splits=1, 
                                          test_size=0.3, 
                                          random_state=42)

train_idx, test_idx = next(strat_shuf_split.split(df[feature_cols], df.Survived))

# Create the dataframes
X_train = df.loc[train_idx, feature_cols]
y_train = df.loc[train_idx, 'Survived']

X_test  = df.loc[test_idx, feature_cols]
y_test  = df.loc[test_idx, 'Survived']


In [102]:
y_train.value_counts(normalize=True)

0    0.616372
1    0.383628
Name: Survived, dtype: float64

In [103]:
y_test.value_counts(normalize=True)

0    0.615672
1    0.384328
Name: Survived, dtype: float64

We can observe that the target feature in the training and testing set have similar proportion of positive and negative labels.

## Logistic Regression

In [104]:
from sklearn.linear_model import LogisticRegression



# Standard logistic regression
lr = LogisticRegression(solver='liblinear')
from sklearn.linear_model import LogisticRegression

In [105]:
from sklearn.linear_model import LogisticRegressionCV

# L1 regularized logistic regression
lr_l1 = LogisticRegressionCV(Cs=10, cv=4, penalty='l1', solver='liblinear').fit(X_train, y_train)


In [106]:
# L2 regularized logistic regression
lr_l2 = LogisticRegressionCV(Cs=10, cv=4, penalty='l2', solver='liblinear').fit(X_train, y_train)

In [107]:
y_pred = list()
coeff_labels = ['lr', 'Lasso(l1)', 'Ridge(l2)']
coeff_models = [lr, lr_l1, lr_l2]

for lab,mod in zip(coeff_labels, coeff_models):
    y_pred.append(pd.Series(mod.predict(X_test), name=lab))
    
    
y_pred = pd.concat(y_pred, axis=1)


y_pred.head()

,lr,Lasso(l1),Ridge(l2)
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [108]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.metrics import  classification_report, f1_score
from sklearn.preprocessing import label_binarize

metrics = list()
cm = dict()

for lab in coeff_labels:

    # Preciision, recall, f-score from the multi-class support function
    precision, recall, f1score, _ = score(y_test, y_pred[lab], average='weighted')
    
    # The usual way to calculate accuracy
    accuracy = accuracy_score(y_test, y_pred[lab])
    
    # ROC-AUC scores can be calculated by binarizing the data
    auc = roc_auc_score(label_binarize(y_test, classes=[0,1]),
              label_binarize(y_pred[lab], classes=[0,1]), 
              average='weighted')
    
    # Last, the confusion matrix
    cm[lab] = confusion_matrix(y_test, y_pred[lab])
    
    metrics.append(pd.Series({'precision':precision, 'recall':recall, 
                              'f1score':f1score, 'accuracy':accuracy,
                              'auc':auc}, 
                             name=lab))

metrics = pd.concat(metrics, axis=1)

In [109]:
metrics

,lr,Lasso(l1),Ridge(l2)
precision,0.815911,0.811926,0.815911
recall,0.817164,0.813433,0.817164
f1score,0.814471,0.810936,0.814471
accuracy,0.817164,0.813433,0.817164
auc,0.794969,0.791939,0.794969


We can see that all three variants of linear regression done well on the test data.

## K Nearest Neighbors:

In [110]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import  classification_report, f1_score
knn = KNeighborsClassifier(n_neighbors=3)
knn = knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
# Preciision, recall, f-score from the multi-class support function
print(classification_report(y_test, y_pred))
print('Accuracy score: ', round(accuracy_score(y_test, y_pred), 4))
print('F1 Score: ', round(f1_score(y_test, y_pred), 4))
print('auc: ', round((roc_auc_score(label_binarize(y_test, classes=[0,1]),
              label_binarize(y_pred, classes=[0,1]), 
              average='weighted')), 4))

              precision    recall  f1-score   support

           0       0.80      0.86      0.83       165
           1       0.75      0.66      0.70       103

    accuracy                           0.78       268
   macro avg       0.77      0.76      0.77       268
weighted avg       0.78      0.78      0.78       268

Accuracy score:  0.7836
F1 Score:  0.701
auc:  0.7604


In [112]:
knn1 = KNeighborsClassifier(n_neighbors=5)
knn1 = knn1.fit(X_train, y_train)
y_pred1 = knn1.predict(X_test)
# Preciision, recall, f-score from the multi-class support function
print(classification_report(y_test, y_pred1))
print('Accuracy score: ', round(accuracy_score(y_test, y_pred1), 4))
print('F1 Score: ', round(f1_score(y_test, y_pred1), 4))
print('auc: ', round((roc_auc_score(label_binarize(y_test, classes=[0,1]),
              label_binarize(y_pred1, classes=[0,1]), 
              average='weighted')), 4))

              precision    recall  f1-score   support

           0       0.81      0.84      0.82       165
           1       0.73      0.68      0.70       103

    accuracy                           0.78       268
   macro avg       0.77      0.76      0.76       268
weighted avg       0.78      0.78      0.78       268

Accuracy score:  0.7799
F1 Score:  0.7035
auc:  0.761


In [113]:
knn2 = KNeighborsClassifier(n_neighbors=7)
knn2 = knn2.fit(X_train, y_train)
y_pred2 = knn2.predict(X_test)
# Preciision, recall, f-score from the multi-class support function
print(classification_report(y_test, y_pred2))
print('Accuracy score: ', round(accuracy_score(y_test, y_pred2), 4))
print('F1 Score: ', round(f1_score(y_test, y_pred2), 4))
print('auc: ', round((roc_auc_score(label_binarize(y_test, classes=[0,1]),
              label_binarize(y_pred2, classes=[0,1]), 
              average='weighted')), 4))

              precision    recall  f1-score   support

           0       0.82      0.84      0.83       165
           1       0.74      0.71      0.72       103

    accuracy                           0.79       268
   macro avg       0.78      0.78      0.78       268
weighted avg       0.79      0.79      0.79       268

Accuracy score:  0.791
F1 Score:  0.7228
auc:  0.7756


## Support Vector Machines:

In [114]:
from sklearn.kernel_approximation import Nystroem
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

svm = SVC()
svm = svm.fit(X_train, y_train)
y_pred =svm.predict(X_test)
# Preciision, recall, f-score from the multi-class support function
print(classification_report(y_test, y_pred))
print('Accuracy score: ', round(accuracy_score(y_test, y_pred), 4))
print('F1 Score: ', round(f1_score(y_test, y_pred), 4))
print('auc: ', round((roc_auc_score(label_binarize(y_test, classes=[0,1]),
              label_binarize(y_pred, classes=[0,1]), 
              average='weighted')), 4))

              precision    recall  f1-score   support

           0       0.83      0.89      0.86       165
           1       0.80      0.70      0.75       103

    accuracy                           0.82       268
   macro avg       0.81      0.79      0.80       268
weighted avg       0.82      0.82      0.81       268

Accuracy score:  0.8172
F1 Score:  0.7461
auc:  0.795


In [115]:
svm.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [125]:
svm1 = SVC(C = 5.0)
svm1 = svm1.fit(X_train, y_train)
y_pred1 =svm1.predict(X_test)
# Preciision, recall, f-score from the multi-class support function
print(classification_report(y_test, y_pred1))
print('Accuracy score: ', round(accuracy_score(y_test, y_pred1), 4))
print('F1 Score: ', round(f1_score(y_test, y_pred1), 4))
print('auc: ', round((roc_auc_score(label_binarize(y_test, classes=[0,1]),
              label_binarize(y_pred1, classes=[0,1]), 
              average='weighted')), 4))

              precision    recall  f1-score   support

           0       0.81      0.90      0.85       165
           1       0.81      0.66      0.73       103

    accuracy                           0.81       268
   macro avg       0.81      0.78      0.79       268
weighted avg       0.81      0.81      0.81       268

Accuracy score:  0.8097
F1 Score:  0.7273
auc:  0.7816


## RandomForest:

If we have M features, a popular method to determine max_features is to use the square root of M, hence max_features = square root of 24 approximately 5.

In [126]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_features=5,n_estimators=20, random_state=0)
rf = rf.fit(X_train, y_train)
y_pred =rf.predict(X_test)
# Preciision, recall, f-score from the multi-class support function
print(classification_report(y_test, y_pred))
print('Accuracy score: ', round(accuracy_score(y_test, y_pred), 4))
print('F1 Score: ', round(f1_score(y_test, y_pred), 4))
print('auc: ', round((roc_auc_score(label_binarize(y_test, classes=[0,1]),
              label_binarize(y_pred, classes=[0,1]), 
              average='weighted')), 4))

              precision    recall  f1-score   support

           0       0.81      0.88      0.85       165
           1       0.78      0.68      0.73       103

    accuracy                           0.80       268
   macro avg       0.80      0.78      0.79       268
weighted avg       0.80      0.80      0.80       268

Accuracy score:  0.8022
F1 Score:  0.7254
auc:  0.7792


In [129]:
rf1 = RandomForestClassifier(max_features=5,n_estimators=50, random_state=0)
rf1 = rf1.fit(X_train, y_train)
y_pred1 =rf1.predict(X_test)
# Preciision, recall, f-score from the multi-class support function
print(classification_report(y_test, y_pred1))
print('Accuracy score: ', round(accuracy_score(y_test, y_pred1), 4))
print('F1 Score: ', round(f1_score(y_test, y_pred1), 4))
print('auc: ', round((roc_auc_score(label_binarize(y_test, classes=[0,1]),
              label_binarize(y_pred1, classes=[0,1]), 
              average='weighted')), 4))

              precision    recall  f1-score   support

           0       0.82      0.87      0.84       165
           1       0.77      0.69      0.73       103

    accuracy                           0.80       268
   macro avg       0.79      0.78      0.79       268
weighted avg       0.80      0.80      0.80       268

Accuracy score:  0.8022
F1 Score:  0.7282
auc:  0.781


## Conclusion:

We can see that LogisticRegression with Ridge regularization is performing better on the given data.

## Submission:

Hence I'm training the logistic regression with entire train_data to furher improve the score.

In [130]:
X = df[feature_cols]
y = df['Survived']
model = LogisticRegressionCV(Cs=10, cv=4, penalty='l2', solver='liblinear').fit(X, y)


In [131]:

features1 = ["Pclass", "Sex", "SibSp", "Parch","Age-binned","Fare-binned", "Embarked"]

df1 = pd.get_dummies(test_data[features1])
X_test1 = df1[feature_cols]
X_test1.head()



,Pclass,SibSp,Parch,Sex_female,Sex_male,Age-binned_8,Age-binned_16,Age-binned_24,Age-binned_32,Age-binned_40,...,Age-binned_80,Fare-binned_1,Fare-binned_2,Fare-binned_3,Fare-binned_4,Fare-binned_5,Fare-binned_6,Embarked_C,Embarked_Q,Embarked_S
0,3,0,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
1,3,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,2,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,3,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
4,3,1,1,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1


In [132]:
predictions = model.predict(X_test1)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
